In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ubfc_datatape_450x32x32_train.h5"
valid_tape = "Z:/rppg/ubfc_datatape_450x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train).cache(), to_tf(valid).cache()

In [3]:
m4 = M_1()
m4.compile(optimizer='Nadam', loss='mae')
m4.build(input_shape=(None, 450, 32, 32, 3))
print(f'Flops per frame:{get_flops(m4, input_sig=[tf.TensorSpec([1, 450, 32, 32, 3])])/450:.0f}')
m4.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:633997
Model: "m_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 450, 64)           196672    
_________________________________________________________________
spectral_transform (Spectral multiple                  82560     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  41024     
_________________________________________________________________
sequential_2 (Sequential)    (None, 450, 64)           256       
_________________________________________________________________
spectral_transform_1 (Spectr multiple                  49792     
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  10272     
_________________

In [ ]:
m4.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/m4_ubfc.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
m4.load_weights('../weights/m4_ubfc.h5')

In [4]:
m4.load_weights('../weights/m4_ubfc.h5')

In [7]:
eval_on_dataset(test_set_CCNU, m4, 450, (32, 32), step=1, batch=32, save='../results/ours_32_UBFC_CCNU.h5')
get_metrics('../results/ours_32_UBFC_CCNU.h5', use_filter=False)

100%|██████████| 179/179 [12:06<00:00,  4.06s/it]


{'Sliding window': {'MAE': 13.004, 'RMSE': 22.901, 'R': 0.16404},
 'Whole video': {'MAE': 13.182, 'RMSE': 23.883, 'R': 0.06188}}

In [6]:
eval_on_dataset(test_set_CCNU_rPPG, m4, 450, (32, 32), step=1, batch=1, save='../results/ours_32_UBFC_CCNU_rPPG.h5')
get_metrics('../results/ours_32_UBFC_CCNU_rPPG.h5', use_filter=False)

100%|██████████| 54/54 [02:28<00:00,  2.75s/it]


{'Sliding window': {'MAE': 11.824, 'RMSE': 20.838, 'R': 0.20572},
 'Whole video': {'MAE': 9.412, 'RMSE': 17.02, 'R': 0.22286}}

In [5]:
eval_on_dataset(test_set_PURE, m4, 450, (32, 32), step=1, batch=1, save='../results/ours_32_UBFC_PURE.h5')
get_metrics('../results/ours_32_UBFC_PURE.h5', use_filter=False)

100%|██████████| 59/59 [01:33<00:00,  1.59s/it]


{'Sliding window': {'MAE': 16.905, 'RMSE': 29.108, 'R': 0.45657},
 'Whole video': {'MAE': 14.481, 'RMSE': 25.992, 'R': 0.56911}}